In [2]:
# !pip install fosforml

In [2]:
# pip show fosforml

## Sklearn Models Registration

### session creation

In [21]:
from fosforml.model_manager.snowflakesession import get_session, get_connection_params
session = get_session()
connection_params = get_connection_params()

### modeling

#### Multiclass classification

In [7]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_iris() 
feature_names = m_dataset.feature_names
u_features = [col.replace(" ","_").replace("(","").replace(")","") for col in feature_names]
m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target


x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

## Custom Function

In [11]:
from snowflake.ml.model import custom_model
from snowflake.ml.model.custom_model import ModelContext

mc = custom_model.ModelContext(
    models={
        'm1': model
    }
)

In [17]:
class ExamplePipelineModel(custom_model.CustomModel):
    def __init__(self, context: ModelContext) -> None:
        super().__init__(context)
        
    @custom_model.inference_api
    def predict(self, input_data: pd.DataFrame) -> pd.DataFrame:
        model_output = self.context.model_ref('m1').predict(input_data)
        return pd.DataFrame({
            'output': model_output })

In [19]:
my_model_pipeline = ExamplePipelineModel(mc)
my_model_pipeline.predict(x_test).head()

output
0       2
1       0
2       0
3       0
4       0

In [32]:
!env | grep connection_details

|=grep connection_details
connection_details={"user": "SAURABHKULKARNI", "password": "Epsilon0*", "account": "fya62509.us-east-1", "role": "SAURABHKULKARNI", "warehouse": "FOSFOR_INSIGHT_WH", "database": "INSIGHT_DESIGNER_SPCS", "schema": "INSIGHT_SPCS_SCHEMA"}


In [26]:
from snowflake.ml.registry import Registry
reg = Registry(session=session, database_name="INSIGHT_DESIGNER_SPCS", schema_name="INSIGHT_SPCS_SCHEMA")
mv = reg.log_model(my_model_pipeline,
            model_name="my_custom_model_pipeline",
            version_name="v1",
            conda_dependencies=["scikit-learn"],
            comment="My Custom ML Model Pipeline",
            sample_input_data=x_train)

In [34]:
reg.show_models()

created_on  \
0 2024-07-26 08:29:19.076000+00:00   
1 2024-07-25 13:18:25.972000+00:00   
2 2024-07-24 13:15:39.020000+00:00   
3 2024-07-25 17:33:56.053000+00:00   
4 2024-07-25 06:30:17.879000+00:00   
5 2024-07-23 17:24:26.333000+00:00   
6 2024-07-25 21:33:31.823000+00:00   
7 2024-07-24 09:53:29.637000+00:00   
8 2024-07-29 13:09:11.826000+00:00   
9 2024-07-30 06:29:29.359000+00:00   

                                                name  model_type  \
0  MODEL_1A4F0336_D3DC_4C10_98BD_FE78E4767E82_FDC...  USER_MODEL   
1  MODEL_6A8513F2_F037_4B45_8F1E_6FB2135256F2_FDC...  USER_MODEL   
2  MODEL_6A8513F2_F037_4B45_8F1E_6FB2135256F2_FDC...  USER_MODEL   
3  MODEL_6A8513F2_F037_4B45_8F1E_6FB2135256F2_FDC...  USER_MODEL   
4  MODEL_6A8513F2_F037_4B45_8F1E_6FB2135256F2_FDC...  USER_MODEL   
5  MODEL_6A8513F2_F037_4B45_8F1E_6FB2135256F2_FDC...  USER_MODEL   
6  MODEL_6A8513F2_F037_4B45_8F1E_6FB2135256F2_FDC...  USER_MODEL   
7  MODEL_8CF5BE83_DDDB_4682_86F1_5DD84583E2D1_FDC...  USER_MODEL   
8  MODEL_CCA393BC_BCBF_4105_9531_9D56DFBD6335_FDC...  USER_MODEL   
9                           MY_CUSTOM_MODEL_PIPELINE  USER_MODEL   

           database_name          schema_name                       comment  \
0  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA       This is a sklearn model   
1  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA  This is a test sklearn model   
2  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA     This is a Snowflake model   
3  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA       This is a sklearn model   
4  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA  This is a test sklearn model   
5  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA  This is a test sklearn model   
6  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA     This is a Snowflake model   
7  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA          This is a test model   
8  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA       This is a sklearn model   
9  INSIGHT_DESIGNER_SPCS  INSIGHT_SPCS_SCHEMA                          None   

             owner default_version_name  \
0  SAURABHKULKARNI                   V1   
1  SAURABHKULKARNI                   V1   
2  SAURABHKULKARNI                   V1   
3  SAURABHKULKARNI                   V1   
4  SAURABHKULKARNI                   V1   
5  SAURABHKULKARNI                   V1   
6  SAURABHKULKARNI                   V1   
7  SAURABHKULKARNI                   V1   
8  SAURABHKULKARNI                   V1   
9  SAURABHKULKARNI                   V1   

                                            versions  \
0                         ["V1","V2","V3","V4","V5"]   
1  ["V1","V10","V11","V12","V13","V14","V15","V16...   
2     ["V1","V2","V3","V4","V5","V6","V7","V8","V9"]   
3  ["V1","V10","V11","V12","V13","V14","V15","V16...   
4                                             ["V1"]   
5                                        ["V1","V2"]   
6                                        ["V1","V2"]   
7                                             ["V1"]   
8                                             ["V1"]   
9                                             ["V1"]   

                                      aliases  
0   {"DEFAULT":"V1","FIRST":"V1","LAST":"V5"}  
1  {"DEFAULT":"V1","FIRST":"V1","LAST":"V27"}  
2   {"DEFAULT":"V1","FIRST":"V1","LAST":"V9"}  
3  {"DEFAULT":"V1","FIRST":"V1","LAST":"V43"}  
4   {"DEFAULT":"V1","FIRST":"V1","LAST":"V1"}  
5   {"DEFAULT":"V1","FIRST":"V1","LAST":"V2"}  
6   {"DEFAULT":"V1","FIRST":"V1","LAST":"V2"}  
7   {"DEFAULT":"V1","FIRST":"V1","LAST":"V1"}  
8   {"DEFAULT":"V1","FIRST":"V1","LAST":"V1"}  
9   {"DEFAULT":"V1","FIRST":"V1","LAST":"V1"}

In [27]:
output_df = mv.run(session.create_dataframe(x_train))

In [29]:
output_df.to_pandas()

sepal_length_cm  sepal_width_cm  petal_length_cm  petal_width_cm  output
0                4.7             3.2              1.6             0.2       0
1                5.5             2.4              3.7             1.0       1
2                5.1             3.7              1.5             0.4       0
3                5.2             3.5              1.5             0.2       0
4                7.2             3.6              6.1             2.5       2
..               ...             ...              ...             ...     ...
107              5.5             2.5              4.0             1.3       1
108              5.0             3.4              1.5             0.2       0
109              6.4             3.1              5.5             1.8       2
110              5.6             2.8              4.9             2.0       2
111              5.2             3.4              1.4             0.2       0

[112 rows x 5 columns]

## Model registration

In [20]:
from fosforml import register_model
register_model(
  model_obj=my_model_pipeline,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="RandomMulticlassMOdelForTesting",
  dataset_source="Snowflake",
  name="RandomMultieefefefclassMOdelForTesting16July",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Exception: Failed to get model performance metrics. 'feature_names'